In [3]:
!ls -la ~/Data/Instacart

total 732888
drwxr-xr-x 3 ike ike      4096 Feb 15 23:16 .
drwxr-xr-x 3 ike ike      4096 Feb 10 13:45 ..
-rw-rw-rw- 1 ike ike      2603 Jun 10  2017 aisles.csv
-rw-rw-rw- 1 ike ike       270 Jun 10  2017 departments.csv
-rw-r--r-- 1 ike ike  27681062 Feb 15 23:17 log.prior.small.csv
-rw-r--r-- 1 ike ike   1191183 Feb 15 23:17 log.train.small.csv
drwxr-xr-x 3 ike ike      4096 Feb 10 21:57 mk
-rw-rw-rw- 1 ike ike 577550706 Jun 10  2017 order_products__prior.csv
-rw-rw-rw- 1 ike ike  24680147 Jun 10  2017 order_products__train.csv
-rw-rw-rw- 1 ike ike 108968645 Jun 10  2017 orders.csv
-rw-r--r-- 1 ike ike   5238924 Feb 15 23:16 orders.small.csv
-rw-rw-rw- 1 ike ike   2166953 Jun 10  2017 products.csv
-rw-rw-rw- 1 ike ike   1475693 Jun 10  2017 sample_submission.csv
-rw-r--r-- 1 ike ike   1475693 Feb 15 23:17 sample_submission.small.csv


In [4]:
!rm ~/Data/Instacart/mk -rf

In [1]:
import pandas as pd
import numpy as np
from setting import *

orders = pd.read_csv(DATA + '/orders.small.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32}, engine='c').\
        rename(columns = {'order_dow': 'order_week', 'order_hour_of_day': 'order_hour', 'days_since_prior_order': 'order_gap'})

In [2]:
log=pd.read_csv(DATA + '/log.train.small.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8}, usecols=['order_id', 'product_id', 'reordered'], engine='c' )
log.sort_values(by=['order_id', 'product_id'], inplace=True)
log = log.reset_index(drop=True)

In [5]:
log_extended = log.merge(orders, on='order_id').drop(columns=['reordered', 'eval_set', 'order_number', 'order_gap'])

In [8]:
log_extended.head()

,order_id,product_id,user_id,order_week,order_hour,order_timeslot,order_timeslot_week
0,98,329,56463,3,8,1,13
1,98,790,56463,3,8,1,13
2,98,1939,56463,3,8,1,13
3,98,3880,56463,3,8,1,13
4,98,4357,56463,3,8,1,13


In [7]:
log_extended['order_timeslot'] = log_extended.order_hour//6
log_extended['order_timeslot_week'] = 4*log_extended.order_week+log_extended.order_timeslot

In [21]:
i1 = log_extended.groupby(by=['product_id', 'order_hour']).size().rename('item_hour_cnt').reset_index().set_index('product_id')

In [22]:
i1['item_hour_prob'] = i1.item_hour_cnt / i1.groupby('product_id')['item_hour_cnt'].sum()

In [24]:
i2 = log_extended.drop_duplicates(['user_id', 'product_id', 'order_hour']).\
groupby(by=['product_id', 'order_hour']).size().\
rename('itemperuser_hour_cnt').reset_index().set_index('product_id')
i2['itemperuser_hour_prob'] = i2.itemperuser_hour_cnt / i2.groupby('product_id')['itemperuser_hour_cnt'].sum()

In [27]:
i3 = log_extended.groupby(by=['product_id', 'order_week']).size().rename('item_week_cnt').\
reset_index().set_index('product_id')
i3['item_week_prob'] = i3.item_week_cnt / i3.groupby('product_id')['item_week_cnt'].sum()
i4 = log_extended.drop_duplicates(['user_id', 'product_id', 'order_week']).\
groupby(by=['product_id', 'order_week']).size().\
rename('itemperuser_week_cnt').reset_index().set_index('product_id')
i4['itemperuser_week_prob'] = i4.itemperuser_week_cnt / \
                              i4.groupby('product_id')['itemperuser_week_cnt'].sum()

In [31]:
i5 = log_extended.groupby(by=['product_id', 'order_timeslot']).size().rename('item_timeslot_cnt').\
reset_index().set_index('product_id')
i5['item_timeslot_prob'] = i5.item_timeslot_cnt / i5.groupby('product_id')['item_timeslot_cnt'].sum()
i6 = log_extended.drop_duplicates(['user_id', 'product_id', 'order_timeslot']).\
groupby(by=['product_id', 'order_timeslot']).size().\
rename('itemperuser_timeslot_cnt').reset_index().set_index('product_id')
i6['itemperuser_timeslot_prob'] = i6.itemperuser_timeslot_cnt / \
                              i6.groupby('product_id')['itemperuser_timeslot_cnt'].sum()

In [32]:
i7 = log_extended.groupby(by=['product_id', 'order_timeslot_week']).size().rename('item_timeslot_week_cnt').\
reset_index().set_index('product_id')
i7['item_timeslot_week_prob'] = i7.item_timeslot_week_cnt / i7.groupby('product_id')['item_timeslot_week_cnt'].sum()
i8 = log_extended.drop_duplicates(['user_id', 'product_id', 'order_timeslot_week']).\
groupby(by=['product_id', 'order_timeslot_week']).size().\
rename('itemperuser_timeslot_week_cnt').reset_index().set_index('product_id')
i8['itemperuser_timeslot_week_prob'] = i8.itemperuser_timeslot_week_cnt / \
                              i8.groupby('product_id')['itemperuser_timeslot_week_cnt'].sum()

In [47]:
i9 = pd.crosstab(log_extended.product_id, log_extended.order_timeslot_week, normalize='index').add_prefix('item_timeslot_week_prob_').reset_index()

In [48]:
i0 = log_extended.\
merge(i1.reset_index(), on=['product_id', 'order_hour']).\
merge(i2.reset_index(), on=['product_id', 'order_hour']).\
merge(i3.reset_index(), on=['product_id', 'order_week']).\
merge(i4.reset_index(), on=['product_id', 'order_week']).\
merge(i5.reset_index(), on=['product_id', 'order_timeslot']).\
merge(i6.reset_index(), on=['product_id', 'order_timeslot']).\
merge(i7.reset_index(), on=['product_id', 'order_timeslot_week']).\
merge(i8.reset_index(), on=['product_id', 'order_timeslot_week']).\
merge(i9, on='product_id')

In [54]:
pd.merge(i1.reset_index(), i2.reset_index(), on=['product_id', 'order_hour']).to_csv(DATA+'/item_hour.csv', index=False)
pd.merge(i3.reset_index(), i4.reset_index(), on=['product_id', 'order_week']).to_csv(DATA+'/item_week.csv', index=False)
pd.merge(i5.reset_index(), i6.reset_index(), on=['product_id', 'order_timeslot']).to_csv(DATA+'/item_timeslot.csv', index=False)
pd.merge(i7.reset_index(), i8.reset_index(), on=['product_id', 'order_timeslot_week']).to_csv(DATA+'/item_timeslot_week.csv', index=False)
i9.to_csv(DATA+'/item_timeslot_week_dist.csv', index=False)

In [50]:
i8

,order_timeslot_week,itemperuser_timeslot_week_cnt,itemperuser_timeslot_week_prob
product_id,,,
1,5,1,0.500000
1,23,1,0.500000
8,26,1,1.000000
10,11,1,0.250000
10,14,1,0.250000
10,18,1,0.250000
10,22,1,0.250000
18,2,1,1.000000
23,0,1,0.500000
